## Librerias

In [399]:
from IPython.display import display, Markdown
import pandas as pd
import random
random.seed(42)
from datetime import datetime, timedelta
from typing import List, Tuple

In [400]:
!pip install faker
import faker
fake = faker.Faker()

In [401]:
!pip install psycopg2-binary
!pip install sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

## Coneccion a la base de datos

In [402]:
# Connect to the database and populate with fake data

host = "localhost"
port = 5432
user = "catedra"
password = "S3cret"
database = "postgres"
driver = "postgresql"

pg = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database))

Session = sessionmaker(bind=pg)

## Mercadopago.SQL

### Correr desde archivo

In [403]:
# correr mercadopago.sql
with open("mercadopago.sql") as f:
    sql = f.read()
    pg.execute(text(sql))

## Crear datos de prueba

### Crear identificadores - DNI, CUIT, CVU/CBU, ALIAS

In [404]:
def random_dni():
    return str(random.randint(10**7, 10**8 - 1))


def random_cuit():
    return (
        str(random.randint(10, 99))
        + "-"
        + random_dni()
        + "-"
        + str(random.randint(0, 9))
    )


def random_cu():
    return (
        "0"
        + str(random.randint(0, 999))
        + "0"
        + str(random.randint(0, 9999))
        + str(random.randint(0, 9))
        + str(random.randint(10**12, 10**13 - 1))
    )


def random_alias():
    return ".".join(fake.words(nb=3))


cu = random_cu()
alias = random_alias()
cuit = random_cuit()

display(cu, alias, cuit)


'06540182405839318110459'

'hard.prevent.body'

'41-39958838-2'

### Crear usuarios de prueba

In [405]:
def insert_clave_uniforme(
        cu: str,
        alias: str,
        esVirtual: bool
    ) -> None:
    """Inserta una clave uniforme en la base de datos

    Args:
        cu (str): Clave Uniforme
        alias (str): Alias
        esVirtual (bool): Es virtual o no
    """

    pg.execute("INSERT INTO Clave VALUES (%s, %s, %s)", (cu, alias, esVirtual))


def create_identifiers() -> Tuple[str, str, str]:
    """Crea identificadores aleatorios

    Returns:
        Tuple[str, str, str]: CU, Alias, CUIT
    """

    cu = random_cu()
    alias = random_alias()
    cuit = random_cuit()

    return cu, alias, cuit


def create_usuario() -> Tuple[str, str, str, str, str, str, str, str]:
    """Crea un usuario aleatorio

    Returns:
        Tuple[str, str, str, str, str, str, str, str]: CVU, Alias, CUIT, Email, Nombre, Apellido, Username, Password
    """

    cvu, alias, cuit = create_identifiers()
    insert_clave_uniforme(cvu, alias, True)
    pg.execute(
        "INSERT INTO Usuario VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (
            cvu,
            cuit,
            (email := fake.email()),
            (first_name := fake.first_name()),
            (last_name := fake.last_name()),
            (user_name := fake.user_name()),
            (password := fake.password()),
            0,
            fake.date_this_century(),
        ),
    )
    return cvu, alias, cuit, email, first_name, last_name, user_name, password


def create_cuenta_bancaria() -> Tuple[str, str]:
    """Crea una cuenta bancaria aleatoria

    Returns:
        Tuple[str, str]: CU, Alias
    """
    cvu, alias, _ = create_identifiers()
    insert_clave_uniforme(cvu, alias, False)
    pg.execute("INSERT INTO CuentaBancaria VALUES (%s, %s)", (cvu, fake.company()))
    return cvu, alias


def create_proveedor_servicio() -> Tuple[str, str]:
    """Crea un proveedor de servicio aleatorio

    Returns:
        Tuple[str, str]: CU, Alias
    """
    cvu, alias, _ = create_identifiers()
    insert_clave_uniforme(cvu, alias, False)
    pg.execute(
        "INSERT INTO ProveedorServicio VALUES (%s, %s, %s, %s)",
        (cvu, fake.company(), fake.word(), fake.date_this_century()),
    )
    return cvu, alias


def add_tarjeta(
        cvu : str,
        number : str,
        cvv : str,
        vencimiento : str
    ):
    """Agrega una tarjeta a la base de datos

    Args:
        cvu (str): CVU del usuario al que pertenece la tarjeta
        number (str): Número de tarjeta
        cvv (str): CVV
        vencimiento (str): Fecha de vencimiento
    """

    pg.execute(
        "INSERT INTO Tarjeta VALUES (%s, %s, %s, %s)", (number, vencimiento, cvv, cvu)
    )


def create_tarjeta(cvu : str) -> Tuple[str, str, str]:
    """Crea una tarjeta aleatoria

    Args:
        cvu (str): CVU del usuario al que pertenece la tarjeta

    Returns:
        Tuple[str, str, str]: Número, CVV, Vencimiento
    """
    
    add_tarjeta(
        cvu,
        (num := fake.credit_card_number()),
        (cvv := fake.credit_card_security_code()),
        (vencimiento := str(fake.date_between(start_date="-1y", end_date="+1y"))),
    )
    return num, cvv, vencimiento


cvu1, alias1, cuit1, email1, first_name1, last_name1, user_name1, password1 = create_usuario()
cvu2, alias2, cuit2, email2, first_name2, last_name2, user_name2, password2 = create_usuario()

cvu_serv1, alias_serv1 = create_proveedor_servicio()
cvu_cuenta1, alias_cuenta1 = create_cuenta_bancaria()

display(
    Markdown(
        f"""
# Usuarios
- {first_name1} {last_name1} - {email1} - {user_name1} - {password1} | {cuit1} - {cvu1} - {alias1}
- {first_name2} {last_name2} - {email2} - {user_name2} - {password2} | {cuit2} - {cvu2} - {alias2}

# Proveedores de Servicio
- {alias_serv1} - {cvu_serv1}

# Cuentas Bancarias
- {alias_cuenta1} - {cvu_cuenta1}
"""
    )

)


# Usuarios
- Jennifer Mason - wardcandice@example.org - kristina42 - BY5MRUgF@_ | 13-22575562-3 - 07540167981560157888921 - their.season.include
- John Rojas - rbailey@example.com - renee67 - 44(J1xUb+J | 38-70291817-9 - 02380827998381094320374 - area.look.much

# Proveedores de Servicio
- nothing.front.little - 0284010628437586875258

# Cuentas Bancarias
- year.item.full - 02200551417683367452945


In [406]:
num_tar1, cvv_tar1, venc_tar1 = create_tarjeta(cvu1)
num_tar1, cvv_tar1, venc_tar1

('2299836556547555', '696', '2025-01-23')

### Probar hacer una transacción entre 2 usuarios uno sin saldo

In [407]:
# Try to insert a transaction without enough balance

try:
    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            cvu1,
            cvu2,
            100,
            fake.date_this_century(),
            "Transferencia",
            "Pendiente",
            False,
            None,
            0,
        ),
    )
    
except Exception as e:
    if "Not enough balance" in str(e):
        display(Markdown("## Not enough balance"))

### Función para hacer la transacción debitada de un usuario

In [408]:
def create_transaction_between_users(
    amount: float,
    description: str,
    sender_cu: str = "",
    reciever_cu: str = "",
    sender_alias: str = "",
    reciever_alias: str = "",
    card_number: str = "",
    fecha : datetime = datetime.now(),
) -> int:
    """Debita una cantidad de dinero de un usuario y la transfiere a otro

    Args:
        sender (str): CVU del usuario que envía el dinero
        reciever (str): CVU del usuario que recibe el dinero
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción

    Returns:
        int: Código de la transacción
    """

    if not (sender_cu or sender_alias):
        raise Exception("Invalid Sender - No CU or Alias")

    if not (reciever_cu or reciever_alias):
        raise Exception("Invalid Reciever - No CU or Alias")

    if (sender_cu != ""):
        # Check if sender exists
        if not pg.execute("SELECT clave_uniforme FROM Usuario WHERE clave_uniforme = %s", (sender_cu,)).fetchone():
            raise Exception("Invalid Sender - Not Found")
        # Check if the sender is virtual
        if not pg.execute("SELECT esVirtual FROM Clave WHERE clave_uniforme = %s", (sender_cu,)).fetchone()[0]:
            raise Exception("Invalid Sender - Not Virtual")

    if (sender_alias != "" and sender_cu is ""):
        sender_cu = pg.execute("SELECT clave_uniforme FROM Clave WHERE alias = %s", (sender_alias,)).fetchone()[0]
        if not sender_cu:
            raise Exception("Invalid Sender - Alias not found")

    if (reciever_alias != "" and reciever_cu is ""):
        reciever_cu = pg.execute("SELECT clave_uniforme FROM Clave WHERE alias = %s", (reciever_alias,)).fetchone()[0]
        if not reciever_cu:
            raise Exception("Invalid Reciever - Alias not found")

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            sender_cu,
            reciever_cu,
            amount,
            fecha,
            description,
            "Rechazada" 
                if pg.execute("SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (sender_cu,)).fetchone()[0] < amount 
                else "Pendiente",
            card_number is not "",
            None if card_number == "" else card_number,
            None if card_number == "" else random.randint(5, 10),
        ),
    )

    # El siguiente código se ejecuta directamente como un trigger en la base de datos

    # pg.execute(
    #     "UPDATE Usuario SET saldo = saldo - %s WHERE clave_uniforme = %s",
    #     (amount, sender),
    # )

    # pg.execute(
    #     "UPDATE Usuario SET saldo = saldo + %s WHERE clave_uniforme = %s",
    #     (amount, reciever),
    # )

    return pg.execute("SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1").fetchone()[0]

<>:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/5j/7b3w3dw95qdcll87vxc3yn300000gn/T/ipykernel_28845/2531978323.py:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (sender_alias != "" and sender_cu is ""):
/var/folders/5j/7b3w3dw95qdcll87vxc3yn300000gn/T/ipykernel_28845/2531978323.py:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (reciever_alias != "" and reciever_cu is ""):
/var/folders/5j/7b3w3dw95qdcll87vxc3yn300000gn/T/ipykernel_28845/2531978323.py:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  card_number is not "",


In [409]:
# Try again to insert a transaction without enough balance

try:
    create_transaction_between_users(
        100, "Transferencia", sender_cu=cvu1, reciever_cu=cvu2
    )
except Exception as e:
    if "Not enough balance" in str(e):
        display(Markdown("## Not enough balance"))

### Depositar dinero en una cuenta desde una cuenta de bancaria

In [410]:
def create_transaccion_deposit(
    user: str,
    cbu_cuenta_bancaria: str,
    amount: float,
    description: str,
    fecha: datetime = datetime.now(),
) -> Tuple[str, float]:
    """Deposita dinero en la cuenta de un usuario

    Args:
        user (str): CVU del usuario
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción

    Returns:
        Tuple[str, float]: Código de la transacción, Saldo final
    """

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            cbu_cuenta_bancaria,
            user,
            amount,
            fecha,
            description,
            "Completada",
            False,
            None,
            None,
        ),
    )

    # El siguiente código se ejecuta directamente como un trigger en la base de datos

    # pg.execute(
    #     "UPDATE Usuario SET saldo = saldo + %s WHERE clave_uniforme = %s",
    #     (amount, user),
    # )

    return (
        str(
            pg.execute(
                "SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1"
            ).fetchone()[0]
        ),
        float(
            pg.execute(
                "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (user,)
            ).fetchone()[0]
        ),
    )

In [411]:
codigo_transaccion, saldo_final = create_transaccion_deposit(
    cvu1, cvu_cuenta1, 1000, "Depósito"
)

display(
    Markdown(
        f"## Transacción de depósito\n- Código: {codigo_transaccion}\n- Saldo final: {saldo_final}"
    )
)

## Transacción de depósito
- Código: 3
- Saldo final: 1000.0

### Transferir dinero entre dos cuentas ahora si con saldo

In [412]:
# Transfer money from user 1 to user 2 now that user 1 has enough balance

codigo_transaccion = create_transaction_between_users(
    100, "Transferencia", sender_cu=cvu1, reciever_cu=cvu2
)

display(Markdown(f"## Transacción de transferencia\n- Código: {codigo_transaccion}"))

# Buscar transacciones y mostrar

pg.execute("SELECT * FROM Transaccion WHERE codigo = %s", (codigo_transaccion,)).fetchone()

## Transacción de transferencia
- Código: 4

(4, '07540167981560157888921', '02380827998381094320374', 100.0, datetime.date(2024, 5, 10), 'Transferencia', 'Completada', False, None, None)

In [413]:
# Listar nombres de Servicios

nombres = pg.execute("SELECT nombre_empresa FROM ProveedorServicio").fetchall()

display(Markdown(f"## Nombres de servicios\n- {', '.join([n[0] for n in nombres])}"))

sample_nombre = random.choice(nombres)[0]

display(Markdown(f"## Nombre de servicio aleatorio\n- {sample_nombre}"))

## Nombres de servicios
- Aguilar-Hernandez

## Nombre de servicio aleatorio
- Aguilar-Hernandez

In [414]:
# Pagar un servicio


def create_transaccion_pay_service(
    user: str,
    service_name: str,
    amount: float,
    description: str,
    fecha: datetime = datetime.now(),
) -> Tuple[str, float]:
    """Paga un servicio

    Args:
        user (str): CVU del usuario
        service_name (str): Nombre del servicio
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción
    
    Returns:
        Tuple[str, float]: Código de la transacción, Saldo final
    """

    # Check for balance first
    if pg.execute("SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (user,)).fetchone()[0] < amount:
        raise Exception("Not enough balance")

    # Hallar el cbu del servicio por su nombre
    cbu_servicio = pg.execute(
        "SELECT clave_uniforme FROM ProveedorServicio WHERE nombre_empresa = %s",
        (service_name,),
    ).fetchone()[0]

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            user,
            cbu_servicio,
            amount,
            fecha,
            description,
            "Pendiente",
            False,
            None,
            None,
        ),
    )

    pg.execute(
        "UPDATE Usuario SET saldo = saldo - %s WHERE clave_uniforme = %s",
        (amount, user),
    )

    return (
        str(
            pg.execute(
                "SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1"
            ).fetchone()[0]
        ),
        float(
            pg.execute(
                "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (user,)
            ).fetchone()[0]
        ),
    )


create_transaccion_pay_service(cvu1, sample_nombre, 100, "Pago de servicio")

('5', 900.0)

In [415]:
# Invertir plata en la fecha y generar rendimientos a futuro


def comenzar_inversion(
    cvu : str,
    comienzo_plazo : datetime = datetime.now(),
):  
    """
    CREATE TABLE Rendimiento (
        id SERIAL PRIMARY KEY,
        fecha_pago DATE,
        comienzo_plazo DATE NOT NULL,
        fin_plazo DATE,
        TNA FLOAT,
        monto FLOAT
    );
    """
    # Se pone a invertir el monto total del usuario
    # La fecha del comienzo del plazo es ahora,
    # El fin del plazo es en 1 dia
    # La fecha de pago es el próximo dia hábil
    # La tna es entre 60 y 80
    # El monto es el saldo del usuario

    # Solo se puede tener un rendimiento en un comienzo de plazo dado
    # Si ya hay un rendimiento activo y se va a generar otro, se debe finalizar el anterior

    # Hallar el saldo del usuario
    saldo = pg.execute(
        "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu,)
    ).fetchone()[0]

    # Hallar si hay un rendimiento activo
    rendimiento_activo = pg.execute(
        "SELECT * FROM Rendimiento WHERE comienzo_plazo = CURRENT_DATE"
    ).fetchone()

    if rendimiento_activo:
        # Finalizar el rendimiento activo
        pg.execute(
            "UPDATE Rendimiento SET fin_plazo = CURRENT_DATE WHERE id = %s",
            (rendimiento_activo[0],),
        )

    # Generar un nuevo rendimiento
    tna = random.uniform(60, 80)
    monto = saldo
    fin_plazo = datetime.now() + timedelta(days=1)

    pg.execute(
        """INSERT INTO Rendimiento (comienzo_plazo, fin_plazo, TNA, monto)
        VALUES (%s, %s, %s, %s)""",
        (
            comienzo_plazo,
            fin_plazo,
            tna,
            monto,
        ),
    )

    # Asociar el rendimiento al usuario
    rendimiento_id = pg.execute(
        "SELECT id FROM Rendimiento ORDER BY id DESC LIMIT 1"
    ).fetchone()[0]

    pg.execute(
        "INSERT INTO RendimientoUsuario VALUES (%s, %s)",
        (cvu, rendimiento_id),
    )

    return rendimiento_id

def pagar_rendimientos_activos_usuario(
    cvu : str
) -> List:
    """
    Paga los rendimientos de un usuario

    Args:
        cvu (str): Clave uniforme del usuario

    Returns:
        List: Lista de los montos pagados
    """

    # Hallar los rendimientos no pagados del usuario, es decir sin fecha de pago
    # JOIN con Rendimiento para hallar los datos del rendimiento

    rendimientos = pg.execute(
        """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
        WHERE clave_uniforme = %s AND pago IS FALSE""",
        (cvu,),
    ).fetchall()

    for rendimiento in rendimientos:
        # Pagar el rendimiento
        updated = pg.execute(
            """
            UPDATE Rendimiento 
            SET fecha_pago = CURRENT_DATE WHERE id = %s
            RETURNING id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
            """,
            (rendimiento[0],),
        ).fetchone()

        # Añadir el rendimiento generado al saldo del usuario
        # El rendimiento es el monto * (1 + TNA / 100) * (fin_plazo - comienzo_plazo) / 365

        # rendimiento_generado = rendimiento[5] * (1 + rendimiento[4] / 100) * (rendimiento[3] - rendimiento[2]).days / 365
        rendimiento_generado = rendimiento[5]
        rendimiento_generado *= 1 + rendimiento[4] / 100
        rendimiento_generado *= (rendimiento[3] - rendimiento[2]).days / 365

        pg.execute(
            "UPDATE Usuario SET saldo = saldo + %s WHERE clave_uniforme = %s",
            (rendimiento_generado, cvu),  
        )

    if not rendimientos:
        return []

    # Refetch los rendimientos para mostrar by r[0] for r in rendimientos
    updated_rendimientos = pg.execute(
        """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
        WHERE clave_uniforme = %s AND Rendimiento.id IN %s""",
        (cvu, tuple([r[0] for r in rendimientos])),
    ).fetchall()

    return updated_rendimientos

In [416]:
rendimiento_id = comenzar_inversion(cvu1)

display(Markdown(f"## Rendimiento comenzado\n- ID: {rendimiento_id}"))

rendimientos = pagar_rendimientos_activos_usuario(cvu1)

display(
    Markdown("## Rendimientos pagados" + "\n".join([f"""
- ID: {r[0]}
- Fecha de pago: {r[1]}
- Comienzo del plazo: {r[2]}
- Fin del plazo: {r[3]}
- TNA: {r[4]}
- Monto Invertido: {r[5]}
- Monto Pagado: {r[5] * (1 + r[4] / 100) * (r[3] - r[2]).days / 365}
""" for r in rendimientos])
    )
)

## Rendimiento comenzado
- ID: 1

## Rendimientos pagados
- ID: 1
- Fecha de pago: 2024-05-11
- Comienzo del plazo: 2024-05-10
- Fin del plazo: 2024-05-11
- TNA: 76.1425654654876
- Monto Invertido: 900.0
- Monto Pagado: 4.3432413402449


In [417]:
# Crear un nuevo usuario 3
# depositar 1000,
# comenzar un rendimiento
# interrumpirlo transfiriendole 500 al usuario 1 (Agregar un trigger para que cuando se cree una transacción, si el usuario tiene un rendimiento activo, se finalice y se cree uno nuevo con el monto restante pero que no se pague)
# continuar el rendimiento y pagar los rendimientos

# TLDR; Hacer una transferencia con un rendimiento activo y ver que pasa


cvu3, alias3, cuit3, email3, first_name3, last_name3, user_name3, password3 = create_usuario()

# Mostrar saldo cvu3
display(pg.execute("SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu3,)).fetchone()[0])

create_transaccion_deposit(cvu3, cvu_cuenta1, 1000, "Depósito")

# Mostrar saldo cvu3 después del depósito
display(pg.execute("SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu3,)).fetchone()[0])

rendimiento_id = comenzar_inversion(cvu3)

display(
    pg.execute(
        """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
    FROM RendimientoUsuario
    JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
    WHERE clave_uniforme = %s""",
        (cvu3,),
    ).fetchall()
)


0.0

1000.0

[(2, None, datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 76.5880932850599, 1000.0)]

In [418]:
# Checkear si el usuario tiene un rendimiento activo, es decir fecha de fin_plazo es posterior a la fecha de la transaccion
# Joinear RendimientoUsuario con Rendimiento para obtener el id del rendimiento activo
display(
    pg.execute(
        """
        SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario 
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id 
        WHERE clave_uniforme = %s -- AND fin_plazo > (SELECT fecha FROM Transaccion WHERE CU_Origen = %s ORDER BY fecha DESC LIMIT 1)
        """,
        (cvu3, cvu3),
    ).fetchall()
)

[(2, None, datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 76.5880932850599, 1000.0)]

In [419]:
create_transaction_between_users(500, "Transferencia", sender_cu=cvu3, reciever_cu=cvu1)
# Mostrar saldo cvu3 después de la transferencia
display(
    pg.execute(
        "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu3,)
    ).fetchone()[0]
)

1000.0

In [420]:
display(
    pg.execute(
        """
        SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario 
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id 
        WHERE clave_uniforme = %s
        """,
        (cvu3,),
    ).fetchall()
)

[(2, None, datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 76.5880932850599, 1000.0),
 (3, None, datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 76.5880932850599, 500.0)]

In [421]:
rendimientos = pagar_rendimientos_activos_usuario(cvu3)
# Mostrar saldo cvu3 después de pagar los rendimientos
display(
    pg.execute(
        "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu3,)
    ).fetchone()[0]
)

display(
    Markdown(
        "## Rendimientos pagados"
        + "\n".join(
            [
                f"""
- ID: {r[0]}
- Fecha de pago: {r[1]}
- Comienzo del plazo: {r[2]}
- Fin del plazo: {r[3]}
- TNA: {r[4]}
- Monto Invertido: {r[5]}
- Monto Pagado: {r[5] * (1 + r[4] / 100) * (r[3] - r[2]).days / 365}
"""
                for r in rendimientos
            ]
        )
    )
)

1007.2570449295231

## Rendimientos pagados
- ID: 2
- Fecha de pago: 2024-05-11
- Comienzo del plazo: 2024-05-10
- Fin del plazo: 2024-05-11
- TNA: 76.5880932850599
- Monto Invertido: 1000.0
- Monto Pagado: 4.83802995301534


- ID: 3
- Fecha de pago: 2024-05-11
- Comienzo del plazo: 2024-05-10
- Fin del plazo: 2024-05-11
- TNA: 76.5880932850599
- Monto Invertido: 500.0
- Monto Pagado: 2.41901497650767


In [422]:
pg.execute(
    """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
    FROM RendimientoUsuario
    JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
    WHERE clave_uniforme = %s""",
    (cvu3,),
).fetchall()

[(2, datetime.date(2024, 5, 11), datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 76.5880932850599, 1000.0),
 (3, datetime.date(2024, 5, 11), datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 76.5880932850599, 500.0)]

In [423]:
# Obtener todos los registros de rendimientos de cvu3

rendimientos_pendientes = pg.execute(
    """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
    FROM RendimientoUsuario
    JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
    WHERE clave_uniforme = %s AND fecha_pago IS NULL""",
    (cvu3,),
).fetchall()

display(
    Markdown(
        f"## Rendimientos Pendientes de {alias3}"
        + "\n".join(
            [
                f"""
- ID: {r[0]}
- Fecha de pago: {r[1]}
- Comienzo del plazo: {r[2]}
- Fin del plazo: {r[3]}
- TNA: {r[4]}
- Monto Invertido: {r[5]}
"""
                for r in rendimientos_pendientes
            ]
        )
    )
)

## Rendimientos Pendientes de conference.sea.employee

In [424]:
# Mostrar usuario cvu2

pg.execute("SELECT * FROM Usuario WHERE clave_uniforme = %s", (cvu3,)).fetchone()

('07470752787661160537249', '20-84093639-4', 'natalieneal@example.com', 'Mark', 'Miller', 'wilsoneric', '(634J*vW)p', 1007.2570449295231, datetime.date(2014, 12, 18))

In [425]:
# Crear un nuevo usuario, y pagarle a cvu1 con una tarjeta

cvu4, alias4, cuit4, email4, first_name4, last_name4, user_name4, password4 = (
    create_usuario()
)

create_tarjeta(cvu4)

valor = 100

In [426]:
cvu4, alias4, cuit4, email4, first_name4, last_name4, user_name4, password4

('06330592591803457621458',
 'last.science.left',
 '94-40587988-4',
 'hughesmatthew@example.com',
 'Andrew',
 'Morton',
 'zacharysmith',
 '+2AGWfYn9t')

In [427]:
pg.execute("SELECT * FROM Tarjeta WHERE CU = %s", (cvu4,)).fetchone()

('36935398823497', datetime.date(2024, 10, 9), 616, '06330592591803457621458')

In [428]:
pg.execute("SELECT * FROM Tarjeta").fetchall()

[('2299836556547555', datetime.date(2025, 1, 23), 696, '07540167981560157888921'),
 ('36935398823497', datetime.date(2024, 10, 9), 616, '06330592591803457621458')]

In [429]:
pg.execute(
    """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
    (
        cvu1,
        cvu4,
        valor,
        fake.date_this_century(),
        "Transferencia",
        "Pendiente",
        True,
        num_tar1,
        8,
    ),
)

codigo_transaccion = pg.execute("SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1").fetchone()[0]

pg.execute(
    """
        INSERT INTO TransaccionTarjeta VALUES (%s, %s)
    """,
    (codigo_transaccion, num_tar1),
)

display(Markdown(f"## Transacción de transferencia con tarjeta\n- Código: {codigo_transaccion}"))

## Transacción de transferencia con tarjeta
- Código: 8

### Pagar a un usuario de MercadoPago por su alias

In [430]:
# Pagar a un usuario de MercadoPago por su alias

# Saldo de cvu1 y alias4 antes de la transacción

display(
    f"Saldo de cvu1: {cvu1} antes de la transacción",
    pg.execute(
        "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu1,)
    ).fetchone()[0]
)

display(
    f"Saldo de alias4: {alias4} antes de la transacción",
    pg.execute(
        "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu4,)
    ).fetchone()[0]
)

create_transaction_between_users(100, "Pago de servicio", sender_cu=cvu1, reciever_alias=alias4)

# Saldo de cvu1 y alias4 despues de la transacción

display(
    f"Saldo de cvu1: {cvu1} antes de la transacción",
    pg.execute(
        "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu1,)
    ).fetchone()[0],
)

display(
    f"Saldo de alias4: {alias4} antes de la transacción",
    pg.execute(
        "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu4,)
    ).fetchone()[0],
)

'Saldo de cvu1: 07540167981560157888921 antes de la transacción'

904.3432413402448

'Saldo de alias4: last.science.left antes de la transacción'

0.0

'Saldo de cvu1: 07540167981560157888921 antes de la transacción'

904.3432413402448

'Saldo de alias4: last.science.left antes de la transacción'

0.0

## Batch de Datos para correr las querys

In [431]:
from tqdm import tqdm

In [432]:
# Batch de Datos para correr las querys

# Crear 100 usuarios, 100 cuentas bancarias, 20 servicios, algunos usuarios tienen tarjetas, algunos generan rendimientos
# Simular el paso de 3 meses

new_data = {}
new_data["usrs"] = []
new_data["cbs"] = []
new_data["servs"] = []

for _ in tqdm(range(100), desc="Usuarios"):
    cvu, alias, cuit, email, first_name, last_name, user_name, password = create_usuario()
    new_data["usrs"].append((cvu, alias, cuit, email, first_name, last_name, user_name, password))

    if random.choice([True, False]):
        create_tarjeta(cvu)

for _ in tqdm(range(100), desc="Cuentas Bancarias"):
    cvu, alias = create_cuenta_bancaria()
    new_data["cbs"].append((cvu, alias))

for _ in tqdm(range(20), desc="Servicios"):
    cvu, alias = create_proveedor_servicio()
    new_data["servs"].append((cvu, alias))

# Random try 20 transacciones entre usuarios sin saldo (todavia no se han hecho depósitos)
for _ in tqdm(range(20), desc="Transacciones sin saldo"):
    try:
        create_transaction_between_users(
            random.randint(0, 100),
            "Transferencia",
            sender_cu=random.choice(new_data["usrs"])[0],
            reciever_cu=random.choice(new_data["usrs"])[0],
        )
    except Exception as e:
        if "Not enough balance" in str(e):
            pass


# Simular el paso del tiempo
dias = 120

for days in tqdm(range(dias), desc="Días"):
    today = datetime.now() - timedelta(days=days)

    for usr in new_data["usrs"]:
        if random.choice([True, False]):
            comenzar_inversion(usr[0], today)

    for serv in new_data["servs"]:
        if random.choice([True, False]):
            try: 
                create_transaccion_pay_service(
                    random.choice(new_data["usrs"])[0], 
                    serv[1], 
                    random.randint(10, 10000), "Pago de servicio", today)
            except Exception as e:
                if "Not enough balance" in str(e):
                    pass

    for usr in new_data["usrs"]:
        pagar_rendimientos_activos_usuario(usr[0])

    # Generar transacciones entre usuarios
    for _ in range(random.randint(5, 20)):

        # Para todos los usuarios entrar plata al sistema
        for usr in new_data["usrs"]:
            create_transaccion_deposit(
                usr[0],
                random.choice(new_data["cbs"])[0],
                random.randint(100, 10000), 
                "Depósito", 
                today
            )
        

        # Seleccionar o un alias o un cvu
        sender = random.choice(new_data["usrs"])

        usr2 = random.choice(list(set(new_data["usrs"]) - {sender}))

        alias = ""
        cvu = ""
        if random.random() > 0.8:
            cvu = usr2[0]
        else:
            alias = usr2[1]

        # Seleccionar random si es con tarjeta o no
        card = random.choice(
                    cards[0]
                    if (
                        cards := pg.execute(
                            "SELECT numero FROM Tarjeta where CU = %s", (sender[0],)
                        ).fetchall()
                    )
                    else [""]
                )

        try: 
            create_transaction_between_users(
                random.randint(
                    0,
                    int(
                        pg.execute(
                            "SELECT saldo FROM Usuario WHERE clave_uniforme = %s", (cvu1,)
                        ).fetchone()[0]
                    ),
                ),
                "Transferencia",
                sender_cu=sender[0],
                reciever_alias=alias,
                reciever_cu=cvu,
                card_number=random.choice(
                    random.choice(cards)
                    if (cards := pg.execute("SELECT numero FROM Tarjeta where CU = %s", (sender[0],)).fetchall())
                    else [""]
                ),
                fecha=today,
            )
        except Exception as e:
            if "Not enough balance" in str(e):
                pass

Días: 100%|██████████| 120/120 [11:33<00:00,  5.78s/it]


#### Checkeo para Query 4 de más adelante

In [433]:
# Transfiero mucha plata desde una cuenta nueva sin saldo

create_transaction_between_users(
    123412341234,
    "Transferencia",
    sender_cu=create_usuario()[0],
    reciever_cu=random.choice(new_data["usrs"])[0],
)

147894

In [434]:
# Checkeo que se creo la transacción pero se encuentra rechazada

pg.execute(
    "SELECT * FROM Transaccion T WHERE MONTO = 123412341234 ORDER BY codigo DESC;"
).fetchall()

[(147894, '0898074488274433921936', '04680518019061693624179', 123412341234.0, datetime.date(2024, 5, 10), 'Transferencia', 'Rechazada', False, None, None)]

In [435]:
# Checkeo que el saldo del usuario no se modifico (debería ser 0)

saldo = pg.execute(
    """SELECT saldo FROM Usuario 
    INNER JOIN Transaccion T ON Usuario.clave_uniforme = T.CU_Origen
    WHERE T.MONTO = 123412341234 ORDER BY T.codigo DESC;"""
).fetchone()[0]

saldo, saldo == 0.0

(0.0, True)

In [436]:
for _ in tqdm(range(20), desc="Transacciones sin saldo"):
    create_transaction_between_users(
        123412341234,
        "Transferencia",
        sender_cu=create_usuario()[0],
        reciever_cu=random.choice(new_data["usrs"])[0],
    )

Transacciones sin saldo: 100%|██████████| 20/20 [00:00<00:00, 47.86it/s]


## Queries

In [437]:
diff = """
CREATE OR REPLACE FUNCTION datediff(type VARCHAR, date_from DATE, date_to DATE) RETURNS INTEGER LANGUAGE plpgsql
AS
$$
DECLARE age INTERVAL;
BEGIN
    CASE type
        WHEN 'year' THEN
            RETURN date_part('year', date_to) - date_part('year', date_from);
        WHEN 'month' THEN
            age := age(date_to, date_from);
            RETURN date_part('year', age) * 12 + date_part('month', age);
        WHEN 'day' THEN
            RETURN (date_from - date_to)::int;
    END CASE;
END;
$$;

"""

### 1. Listar los usuarios que realizaron transacciones con tarjeta de crédito

In [438]:
q1 = """
SELECT u.clave_uniforme, u.nombre, u.apellido
FROM Usuario u INNER JOIN Transaccion t ON (u.clave_uniforme = t.CU_Origen)
WHERE t.es_con_tarjeta = TRUE;
"""

q1_res = pd.read_sql(q1, pg)
q1_res

,clave_uniforme,nombre,apellido
0,07540167981560157888921,Jennifer,Mason
1,09210847975359911514519,Peter,Flowers
2,06370816771909502233968,Sarah,Smith
3,02710745449562955360970,Jeffery,Rangel
4,08500897673799877189177,Jacqueline,Willis
...,...,...,...
823,07890214348523518553916,Valerie,Martinez
824,04760723874760998180894,Carlos,Bell
825,06830908643366003539397,Christian,Garcia
826,04060324912792044392593,Marie,Melton


### 2. Listar las transacciones realizadas por un usuario en particular en 2024.

In [439]:
q2 = """
SELECT *
FROM Transaccion t
WHERE EXTRACT(YEAR FROM t.fecha) = 2024
AND t.CU_Origen = (SELECT CU_Origen FROM Transaccion WHERE CU_Origen IS NOT NULL LIMIT 1);
"""

q2_res = pd.read_sql(q2, pg)
q2_res

,codigo,cu_origen,cu_destino,monto,fecha,descripcion,estado,es_con_tarjeta,numero,interes
0,2,07540167981560157888921,02380827998381094320374,100.0,2024-05-10,Transferencia,Rechazada,False,None,None
1,4,07540167981560157888921,02380827998381094320374,100.0,2024-05-10,Transferencia,Completada,False,None,None
2,5,07540167981560157888921,0284010628437586875258,100.0,2024-05-10,Pago de servicio,Completada,False,None,None
3,9,07540167981560157888921,06330592591803457621458,100.0,2024-05-10,Pago de servicio,Completada,False,None,None


### 3. Obtener los rendimientos en un periodo de tiempo para todos los usuarios.

In [440]:
q3 = """
SELECT 
    ru.clave_uniforme, 
    SUM(r.monto * r.TNA * datediff('day', r.fin_plazo::date, r.comienzo_plazo::date) /  365) AS rendimiento_total_mes_actual 
FROM Rendimiento r INNER  JOIN RendimientoUsuario ru ON r.id = ru.id 
WHERE EXTRACT(MONTH  FROM r.fecha_pago) = 5 AND EXTRACT(YEAR FROM r.fecha_pago) = 2024
GROUP BY ru.clave_uniforme;
"""

q3_res = pd.read_sql(diff + q3, pg)
q3_res

,clave_uniforme,rendimiento_total_mes_actual
0,01410834873689120466741,2.010770e+24
1,0150040009010426801904,4.254849e+23
2,01600392922320925748519,7.567886e+24
3,01850959069268203927949,3.989437e+22
4,0186082346756884415347,1.629686e+25
...,...,...
97,09720405131360086746040,1.071851e+23
98,0988084143007801558484,4.506127e+22
99,0990706258230249215505,1.632670e+25
100,0991095892006369269690,7.021341e+24


### 4. Cantidad de transacciones rechazadas y la suma de los montos de las mismas para cada usuario.

In [441]:
q4 = """
SELECT 
	u.clave_uniforme, 
	COUNT(*) AS cantidad, 
	SUM(t.monto) AS monto_total
FROM Transaccion t RIGHT JOIN Usuario u ON (t.CU_origen = u.clave_uniforme)
WHERE t.estado = 'Rechazada'
GROUP BY u.clave_uniforme
HAVING SUM(t.monto) IS NOT NULL;
"""

q4_res = pd.read_sql(q4, pg)
q4_res.head()

,clave_uniforme,cantidad,monto_total
0,010049805895069473283,1,1.234123e+11
1,01170492263454004246155,1,1.234123e+11
2,0150040009010426801904,1,3.900000e+01
3,0186082346756884415347,1,9.200000e+01
4,02310966269470293696301,1,1.234123e+11


### 5. El provedor de servicios con mas pagos recibidos en el último mes.

In [442]:
q5 = """
SELECT ps.clave_uniforme, ps.nombre_empresa, COUNT(*) AS pagos_recibidos, SUM(t.monto) AS monto_total
FROM ProveedorServicio ps INNER JOIN Transaccion t ON (ps.clave_uniforme = t.CU_Destino)
WHERE t.estado = 'Completada'
GROUP BY ps.clave_uniforme, ps.nombre_empresa
ORDER BY pagos_recibidos
LIMIT 1;
"""

q5_res = pd.read_sql(q5, pg)
q5_res

,clave_uniforme,nombre_empresa,pagos_recibidos,monto_total
0,0284010628437586875258,Aguilar-Hernandez,1,100.0


### 6. Calcular la diferencia porcentual de rendimientos respecto al mes anterior para cada usuario.

In [443]:
q6 = """
WITH rendimientos_mes_actual AS ( 
	SELECT 
		ru.clave_uniforme, 
		SUM(r.monto * r.TNA *  datediff('day', r.fin_plazo::date, r.comienzo_plazo::date) /  365) AS rendimiento_total_mes_actual 
	FROM Rendimiento r INNER  JOIN RendimientoUsuario ru ON r.id = ru.id 
	WHERE  EXTRACT(MONTH  FROM r.fecha_pago) =  EXTRACT(MONTH  FROM  CURRENT_DATE) 
	GROUP  BY ru.clave_uniforme 
),
rendimientos_mes_anterior AS ( 
	SELECT 
		ru.clave_uniforme, 
		SUM(r.monto * r.TNA *  datediff('day', r.fin_plazo::date, r.comienzo_plazo::date) /  365) AS rendimiento_total_mes_anterior 
	FROM Rendimiento r INNER  JOIN RendimientoUsuario ru ON r.id = ru.id 
	WHERE  EXTRACT(MONTH  FROM r.fecha_pago) =  EXTRACT(MONTH  FROM  CURRENT_DATE) -  1  
	GROUP  BY ru.clave_uniforme 
) 
SELECT 
	rma.clave_uniforme, 
	rma.rendimiento_total_mes_actual, 
	COALESCE(
        -- rma.rendimiento_total_mes_actual / rmb.rendimiento_total_mes_anterior, division por cero
        CASE 
            WHEN rmb.rendimiento_total_mes_anterior = 0 THEN NULL
            ELSE rma.rendimiento_total_mes_actual / rmb.rendimiento_total_mes_anterior
        END,
        -- rma.rendimiento_total_mes_actual / rma.rendimiento_total_mes_actual,
        CASE 
            WHEN rmb.rendimiento_total_mes_anterior = 0 THEN 1
            ELSE 0
        END,
        -- -rmb.rendimiento_total_mes_anterior / rmb.rendimiento_total_mes_anterior, 
        CASE 
            WHEN rma.rendimiento_total_mes_actual = 0 THEN -1
            ELSE 0
        END,
        0) AS porcentaje_cambio 
FROM rendimientos_mes_actual rma LEFT  JOIN rendimientos_mes_anterior rmb ON rma.clave_uniforme = rmb.clave_uniforme;
"""

q6_res = pd.read_sql(diff + q6, pg)
q6_res

,clave_uniforme,rendimiento_total_mes_actual,porcentaje_cambio
0,01410834873689120466741,2.010770e+24,0.0
1,0150040009010426801904,4.254849e+23,0.0
2,01600392922320925748519,7.567886e+24,0.0
3,01850959069268203927949,3.989437e+22,0.0
4,0186082346756884415347,1.629686e+25,0.0
...,...,...,...
97,09720405131360086746040,1.071851e+23,0.0
98,0988084143007801558484,4.506127e+22,0.0
99,0990706258230249215505,1.632670e+25,0.0
100,0991095892006369269690,7.021341e+24,0.0


### 7. Ranking de usuarios con mayor monto transaccionado en el último mes.

In [444]:
q7 = """
SELECT 
	t.CU_Origen, 
	MAX(t.monto) AS mayor_monto 
FROM Transaccion t
WHERE EXTRACT(MONTH  FROM t.fecha) =  EXTRACT(MONTH  FROM  CURRENT_DATE)
AND t.estado = 'Completada'
GROUP BY t.CU_Origen
ORDER  BY mayor_monto DESC;
"""

q7_res = pd.read_sql(q7, pg)
q7_res.head()

,cu_origen,mayor_monto
0,07640223616980376640166,9999.0
1,07640705527687818928252,9999.0
2,08720874266574670581259,9998.0
3,01000164671807578179167,9997.0
4,0920483558308949776358,9996.0


### 8. Ranking de usuarios por rendimientos obtenidos.

In [445]:
q8 = """
SELECT 
	ru.clave_uniforme, 
	SUM(r.monto * r.TNA * datediff('day', r.fin_plazo::date, r.comienzo_plazo::date) /  365) AS rendimiento_total 
FROM Rendimiento r INNER JOIN RendimientoUsuario ru ON r.id = ru.id 
WHERE r.fin_plazo < CURRENT_DATE
GROUP  BY ru.clave_uniforme 
ORDER  BY rendimiento_total DESC;
"""

q8_res = pd.read_sql(diff + q8, pg)
q8_res.head()

,clave_uniforme,rendimiento_total
0,06200994912899711446822,55417.293629
1,05500349283329011301551,21885.786946
2,07230350518170082261076,16988.084962
3,0510885602644839314307,15617.408803
4,07830441594082739701132,14400.642335


### 9. Calcular el saldo resultante después de cada transaccion

#### Recibida

In [446]:
q9_recibida = """
SELECT 
    t.codigo AS operacion_id,
    t.CU_Destino AS usuario_id,
    t.fecha,
    'Transaccion Recibida' AS tipo_operacion,
    t.monto,
    t.monto + SUM(t2.monto) OVER (PARTITION BY t.CU_Destino ORDER BY t.fecha, t.codigo ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS saldo_resultante
FROM Transaccion t
LEFT JOIN Transaccion t2 ON t.CU_Destino = t2.CU_Destino AND t.fecha >= t2.fecha AND t.codigo >= t2.codigo;
"""

q9_recibida_res = pd.read_sql(q9_recibida, pg)
q9_recibida_res

,operacion_id,usuario_id,fecha,tipo_operacion,monto,saldo_resultante
0,146890,01410834873689120466741,2024-01-12,Transaccion Recibida,8832.0,17664.0
1,146991,01410834873689120466741,2024-01-12,Transaccion Recibida,1052.0,10936.0
2,146991,01410834873689120466741,2024-01-12,Transaccion Recibida,1052.0,19768.0
3,147092,01410834873689120466741,2024-01-12,Transaccion Recibida,123.0,19891.0
4,147092,01410834873689120466741,2024-01-12,Transaccion Recibida,123.0,28723.0
...,...,...,...,...,...,...
1149161,1439,09930662288269068838141,2024-05-10,Transaccion Recibida,4414.0,53945983.0
1149162,1439,09930662288269068838141,2024-05-10,Transaccion Recibida,4414.0,53951266.0
1149163,1439,09930662288269068838141,2024-05-10,Transaccion Recibida,4414.0,53955680.0
1149164,1439,09930662288269068838141,2024-05-10,Transaccion Recibida,4414.0,53964952.0


#### Realizada

In [447]:
q9_realizada = """
SELECT 
    t.codigo AS operacion_id,
    t.CU_Origen AS usuario_id,
    t.fecha,
    'Transaccion Realizada' AS tipo_operacion,
    t.monto * -1 AS monto, -- Se multiplica por -1 para reflejar la disminución del saldo
    (t.monto * -1) + SUM(t2.monto) OVER (PARTITION BY t.CU_Origen ORDER BY t.fecha, t.codigo ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS saldo_resultante
FROM Transaccion t
LEFT JOIN Transaccion t2 ON t.CU_Origen = t2.CU_Origen AND t.fecha >= t2.fecha AND t.codigo >= t2.codigo;
"""

q9_realizada_res = pd.read_sql(q9_realizada, pg)
q9_realizada_res

,operacion_id,usuario_id,fecha,tipo_operacion,monto,saldo_resultante
0,146926,01000164671807578179167,2024-01-12,Transaccion Realizada,-8801.0,0.0
1,146982,01000164671807578179167,2024-01-12,Transaccion Realizada,-3911.0,8801.0
2,146982,01000164671807578179167,2024-01-12,Transaccion Realizada,-3911.0,17602.0
3,147026,01000164671807578179167,2024-01-12,Transaccion Realizada,-8172.0,17252.0
4,147026,01000164671807578179167,2024-01-12,Transaccion Realizada,-8172.0,25424.0
...,...,...,...,...,...,...
1176524,19118,0991095892006369269690,2024-04-26,Transaccion Realizada,-354.0,9909.0
1176525,10533,0991095892006369269690,2024-05-03,Transaccion Realizada,-764.0,10263.0
1176526,8614,0991095892006369269690,2024-05-04,Transaccion Realizada,-738.0,11027.0
1176527,3867,0991095892006369269690,2024-05-08,Transaccion Realizada,-503.0,11765.0


#### Recibidas y Realizadas

In [448]:
q9_rec_real = q9_recibida[:-2] + " UNION ALL " + q9_realizada

q9_rec_real_res = pd.read_sql(q9_rec_real, pg)
q9_rec_real_res

,operacion_id,usuario_id,fecha,tipo_operacion,monto,saldo_resultante
0,146890,01410834873689120466741,2024-01-12,Transaccion Recibida,8832.0,17664.0
1,146991,01410834873689120466741,2024-01-12,Transaccion Recibida,1052.0,10936.0
2,146991,01410834873689120466741,2024-01-12,Transaccion Recibida,1052.0,19768.0
3,147092,01410834873689120466741,2024-01-12,Transaccion Recibida,123.0,19891.0
4,147092,01410834873689120466741,2024-01-12,Transaccion Recibida,123.0,28723.0
...,...,...,...,...,...,...
2325690,19118,0991095892006369269690,2024-04-26,Transaccion Realizada,-354.0,9909.0
2325691,10533,0991095892006369269690,2024-05-03,Transaccion Realizada,-764.0,10263.0
2325692,8614,0991095892006369269690,2024-05-04,Transaccion Realizada,-738.0,11027.0
2325693,3867,0991095892006369269690,2024-05-08,Transaccion Realizada,-503.0,11765.0


### 10. Calcular los intereses ganados en transacciones pagadas con tarjeta en el último mes.

#### Crear 100 pagos con tarjetas en el último mes

In [449]:
# Crear 100 pagos con tarjetas en el último mes

for _ in tqdm(range(100), desc="Pagos con tarjeta"):
    
    usr = random.choice(new_data["usrs"])
    # Crearle una tarjeta
    num, cvv, venc = create_tarjeta(usr[0])

    receiver = random.choice(new_data["usrs"])

    create_transaction_between_users(
        random.randint(10, 1000),
        "Pago con tarjeta",
        sender_cu=usr[0],
        reciever_cu=receiver[0],
        card_number=num,
    )


    

Pagos con tarjeta: 100%|██████████| 100/100 [00:00<00:00, 129.30it/s]


In [450]:
pd.read_sql(
    """
    SELECT *
    FROM Transaccion t
    WHERE t.es_con_tarjeta = TRUE
    """,
    pg,
)

,codigo,cu_origen,cu_destino,monto,fecha,descripcion,estado,es_con_tarjeta,numero,interes
0,8,07540167981560157888921,06330592591803457621458,100.0,2007-06-04,Transferencia,Pendiente,True,2299836556547555,8.0
1,130,09210847975359911514519,09720405131360086746040,93.0,2024-05-10,Transferencia,Pendiente,True,3568401528671306,8.0
2,231,06370816771909502233968,03120931694658293099819,320.0,2024-05-10,Transferencia,Pendiente,True,6011864306770341,9.0
3,433,02710745449562955360970,0150040009010426801904,29.0,2024-05-10,Transferencia,Pendiente,True,4786634236121719,9.0
4,534,08500897673799877189177,07230350518170082261076,848.0,2024-05-10,Transferencia,Pendiente,True,2608939609561015,9.0
...,...,...,...,...,...,...,...,...,...,...
923,148010,07350629175292388824675,0870395465229504111657,68.0,2024-05-10,Pago con tarjeta,Pendiente,True,4669340648810202,7.0
924,148011,08790771139099958073842,08010674533801464598795,131.0,2024-05-10,Pago con tarjeta,Pendiente,True,3543338027526773,9.0
925,148012,0150040009010426801904,07670883704944283045955,498.0,2024-05-10,Pago con tarjeta,Pendiente,True,6011960392320817,5.0
926,148013,09570427235197532457626,0870395465229504111657,992.0,2024-05-10,Pago con tarjeta,Pendiente,True,342622725247207,7.0


In [451]:
q10 = """
SELECT SUM(t.monto * t.interes)
FROM Transaccion t
WHERE t.es_con_tarjeta = TRUE AND t.fecha >= CURRENT_DATE - INTERVAL '1 month';
"""

q10_res = pd.read_sql(q10, pg)
q10_res

,sum
0,1123131.0
